Andy Snitgen  
Professor Karen Jin  
Comp 574 - Applied Computing II  
October 22, 2021  

In [1]:
import requests, json, time, pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup

# Create the data frame
column_names = ['Title', 'Year', 'Actor', 'US Box Office', 'International B.O.']
df = pd.DataFrame(columns = column_names)

In [2]:
# Create a function that grabs the relevant data and returns the new row
def data(movie_title):
    # Set initial API string for IMDB
    api_key = "k_nh5thpq8"
    base_url = "https://imdb-api.com/en/API/SearchMovie/"
    complete_url = base_url + api_key + "/" + movie_title

    # Get the title ID
    payload = requests.get(complete_url).json()
    df_temp = pd.json_normalize(payload['results'])
    id_num = df_temp.id.iloc[0]

    # use the id number to get the required info
    base_url = "https://imdb-api.com/en/API/Title/"
    complete_url = base_url + api_key + "/" + id_num 
    payload = requests.get(complete_url).json()
    df_temp = pd.json_normalize(payload)
    df_temp2 = pd.json_normalize(payload['starList'])

    # Extract relevant data and put it into variables
    title = df_temp.title.iloc[0]
    year = df_temp.year.iloc[0]
    actor = df_temp2.name.iloc[0]
    us_box = df_temp['boxOffice.grossUSA'].iloc[0]
    itl_box = df_temp["boxOffice.cumulativeWorldwideGross"].iloc[0]

    # Check if International Box Office Data exists, if it doesn't replace the value with 'No Data'
    if title == 'No Time to Die':  # No Time to Die has to be put in manually as API doesn't have data yet
        us_box = '$120,357,453'
        itl_box = '$525,482,453'
    us_box_float = floater(us_box)
    itl_box_float =  floater(itl_box)
    if abs((us_box_float - itl_box_float) / us_box_float) <= 0.01:
        itl_box = "No Data"

    # Put variables into main data frame
    new_row = {'Title': title, 'Year': year, 'Actor': actor, 'US Box Office': us_box, "International B.O.": itl_box}
    return new_row
     

In [3]:
# Create a function that converts $ strings into floats
def floater(input_str):
    floater = input_str.strip('$')
    floater = floater.replace(',','')
    floater = float(floater)
    return floater

In [4]:
# Create the list of movies to collect data on
movie_list = \
['dr no',
'from russia with love',
'goldfinger',
'thunderball', 
'you only live twice',
'on her majestys secret service',
'diamonds are forever',
'live and let die',
'the man with the golden gun',
'the spy who loved me',
'moonraker',
'for your eyes only',
'octopussy',
'view to a kill',
'the living daylights',
'licence to kill',
'goldeneye',
'tomorrow never dies',
'the world is not enough',
'die another day',
'casino royale',
'quantum of solace',
'skyfall',
'spectre',
'no time to die']

In [5]:
# These commented out lines are use for debugging
#movie_list = movie_list[24:25] # slices small sections of the list 
#print(movie_list)


# Iterate through the list and add results to data frame 
for movie_title in movie_list:
    print('working on ' + movie_title)
    time.sleep(0.1)  # Put into reduce hang up error on home PC
    new_row = data(movie_title)
    df = df.append(new_row, ignore_index=True)
    
df

working on dr no
working on from russia with love
working on goldfinger
working on thunderball
working on you only live twice
working on on her majestys secret service
working on diamonds are forever
working on live and let die
working on the man with the golden gun
working on the spy who loved me
working on moonraker
working on for your eyes only
working on octopussy
working on view to a kill
working on the living daylights
working on licence to kill
working on goldeneye
working on tomorrow never dies
working on the world is not enough
working on die another day
working on casino royale
working on quantum of solace
working on skyfall
working on spectre
working on no time to die


,Title,Year,Actor,US Box Office,International B.O.
0,Dr. No,1962,Sean Connery,"$16,067,035",No Data
1,From Russia with Love,1963,Sean Connery,"$24,796,765",No Data
2,Goldfinger,1964,Sean Connery,"$51,081,062",No Data
3,Thunderball,1965,Sean Connery,"$63,595,658",No Data
4,You Only Live Twice,1967,Sean Connery,"$43,084,787",No Data
5,On Her Majesty's Secret Service,1969,George Lazenby,"$22,774,493",No Data
6,Diamonds Are Forever,1971,Sean Connery,"$43,819,547",No Data
7,Live and Let Die,1973,Roger Moore,"$35,377,836",No Data
8,The Man with the Golden Gun,1974,Roger Moore,"$20,972,000",No Data
9,The Spy Who Loved Me,1977,Roger Moore,"$46,838,673",No Data


In [38]:
# Calculate the Average change from Domestic to International Box office
total = 0
for index, row in df.iterrows():
    if df['International B.O.'].values[index] != 'No Data':
        usbo = floater(str(df['US Box Office'].values[index]))
        itlbo = floater(str(df['International B.O.'].values[index]))
        total = total + (itlbo/usbo)
counter = df['International B.O.'].str.contains("No Data").value_counts()[0]
change = total/counter

# Apply the average change by multiplying US Box Office by 'change' variable to replace 'No Data'
for index, row in df.iterrows():
    if df['International B.O.'].values[index] == 'No Data':
        usbo = floater(str(df['US Box Office'].values[index]))
        itlbo = usbo*change
        itlbo = "${:,.0f}".format(itlbo)
        df['International B.O.'][index] = itlbo

# Add a new column 'Total Revenue' that consists of the sum of US and International Box Offices
for index, row in df.iterrows():
    total = 0
    usbo = floater(str(df['US Box Office'].values[index]))
    itlbo = floater(str(df['International B.O.'].values[index]))
    total = usbo + itlbo
    total = "${:,.0f}".format(total)    
    df['Total Revenue'][index] = total
df

,Title,Year,Actor,US Box Office,International B.O.,Total Revenue
0,Dr. No,1962,Sean Connery,"$16,067,035","$56,367,173","$72,434,208"
1,From Russia with Love,1963,Sean Connery,"$24,796,765","$86,993,247","$111,790,012"
2,Goldfinger,1964,Sean Connery,"$51,081,062","$179,205,128","$230,286,190"
3,Thunderball,1965,Sean Connery,"$63,595,658","$223,109,458","$286,705,116"
4,You Only Live Twice,1967,Sean Connery,"$43,084,787","$151,152,198","$194,236,985"
5,On Her Majesty's Secret Service,1969,George Lazenby,"$22,774,493","$79,898,612","$102,673,105"
6,Diamonds Are Forever,1971,Sean Connery,"$43,819,547","$153,729,919","$197,549,466"
7,Live and Let Die,1973,Roger Moore,"$35,377,836","$124,114,288","$159,492,124"
8,The Man with the Golden Gun,1974,Roger Moore,"$20,972,000","$73,575,016","$94,547,016"
9,The Spy Who Loved Me,1977,Roger Moore,"$46,838,673","$164,321,768","$211,160,441"


In [87]:
# Scrape inflation data from the internet
page = requests.get('https://www.minneapolisfed.org/about-us/monetary-policy/inflation-calculator/consumer-price-index-1913-')
soup = BeautifulSoup(page.content, 'html.parser')
data = soup.find('tbody')
tables = data.find_all('td')
info_list = []
for elem in tables:
    info = elem.findChildren()[0].contents[0]
    info_list.append(info)

# Put the data into lists
year = []
cpi = []
for value in info_list[::3]:
    value = value.replace('*', '')
    v
    year.append(value)
for value in info_list[1::3]:
    cpi.append(value)
zip_iterator = zip(year, cpi)
inf_dict = dict(zip_iterator)
inf_dict

{'1913': '9.9',
 '1914': '10.0',
 '1915': '10.1',
 '1916': '10.9',
 '1917': '12.8',
 '1918': '15.0',
 '1919': '17.3',
 '1920': '20.0',
 '1921': '17.9',
 '1922': '16.8',
 '1923': '17.1',
 '1924': '17.1',
 '1925': '17.5',
 '1926': '17.7',
 '1927': '17.4',
 '1928': '17.2',
 '1929': '17.2',
 '1930': '16.7',
 '1931': '15.2',
 '1932': '13.6',
 '1933': '12.9',
 '1934': '13.4',
 '1935': '13.7',
 '1936': '13.9',
 '1937': '14.4',
 '1938': '14.1',
 '1939': '13.9',
 '1940': '14.0',
 '1941': '14.7',
 '1942': '16.3',
 '1943': '17.3',
 '1944': '17.6',
 '1945': '18.0',
 '1946': '19.5',
 '1947': '22.3',
 '1948': '24.0',
 '1949': '23.8',
 '1950': '24.1',
 '1951': '26.0',
 '1952': '26.6',
 '1953': '26.8',
 '1954': '26.9',
 '1955': '26.8',
 '1956': '27.2',
 '1957': '28.1',
 '1958': '28.9',
 '1959': '29.2',
 '1960': '29.6',
 '1961': '29.9',
 '1962': '30.3',
 '1963': '30.6',
 '1964': '31.0',
 '1965': '31.5',
 '1966': '32.5',
 '1967': '33.4',
 '1968': '34.8',
 '1969': '36.7',
 '1970': '38.8',
 '1971': '40.5'